# Sandbox
For playing around during development.

In [1]:
%pylab inline
import pysd
print pysd.__version__
print pysd.__file__
import pandas as pd

Populating the interactive namespace from numpy and matplotlib
0.3.2
pysd/__init__.pyc


In [2]:
mdl_file = 'tests/test-models/tests/subscript_3d_arrays/test_subscript_3d_arrays.mdl'
# mdl_file = 'tests/test-models/tests/subscript_2d_arrays/test_subscript_2d_arrays.mdl'
mdl_file = 'tests/test-models/tests/subscript_individually_defined_1_of_2d_arrays/subscript_individually_defined_1_of_2d_arrays.mdl'
model = pysd.read_vensim(mdl_file)

In [3]:
a = model.run()
a.head()

,stock_a
0,"[[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]"
1,"[[1.01, 2.02], [3.03, 4.04], [5.05, 6.06]]"
2,"[[1.02, 2.04], [3.06, 4.08], [5.1, 6.12]]"
3,"[[1.03, 2.06], [3.09, 4.12], [5.15, 6.18]]"
4,"[[1.04, 2.08], [3.12, 4.16], [5.2, 6.24]]"


In [4]:
model.components.stock_a.dimension_directory

{'one_dimensional_subscript': 0, 'second_dimension_subscript': 1}

In [26]:
a = pd.DataFrame(data=1, index=range(5), columns=['hi','there'])
a

,hi,there
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1


In [33]:
a.columns = ['hi', 'mounir']

In [34]:
a

,hi,mounir
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1


In [37]:
b = set(['hi', 'there'])
b.add('mounir')
b

{'hi', 'mounir', 'there'}

In [52]:
c = set(['hi'])
#len(c)
list(c)[0]

'hi'

In [49]:
shape = np.zeros(4)
shape

array([ 0.,  0.,  0.,  0.])

In [39]:


subscript_references = [['one_dimensional_subscript', 'second_dimension_subscript']]

num_dimensions = len(subscript_references[0])
reference_sets = [set() for _ in range(num_dimensions)]
    
for subscript_reference in subscript_references:
    [reference_sets[i].add(element) for i, element in enumerate(subscript_reference)]

print reference_sets

[set(['one_dimensional_subscript']), set(['second_dimension_subscript'])]


In [22]:
a.loc[0]['stock_a'].flatten()

array([ 1.,  2.,  2.,  4.,  3.,  5.,  4.,  3.,  5.,  1.,  6.,  4.])

In [ ]:
func(df, subdict, dimdict)

In [6]:
model.components._subscript_dict

{'one_dimensional_subscript': {'entry_1': 0, 'entry_2': 1, 'entry_3': 2},
 'second_dimension_subscript': {'column_1': 0, 'column_2': 1},
 'third_dimension_subscript': {'depth_1': 0, 'depth_2': 1}}

In [ ]:

#inputs
dataframe of model output

subscript_dict = {'one_dimensional_subscript': {'entry_1': 0, 'entry_2': 1, 'entry_3': 2},
 'second_dimension_subscript': {'column_1': 0, 'column_2': 1},
 'third_dimension_subscript': {'depth_1': 0, 'depth_2': 1}}

dimdict = {'one_dimensional_subscript': 0,
 'second_dimension_subscript': 1,
 'third_dimension_subscript': 2}

#output
#flattened dataframe of output with names akin to: 
    Rate A[Entry 1,Column 1]	Rate A[Entry 1,Column 2]	Rate A[Entry 2,Column 1]	Rate A[Entry 2,Column 2]	Rate A[Entry 3,Column 1]	Rate A[Entry 3,Column 2]

In [ ]:
list(['stock_a__entry_1__column_2__depth_2'])

In [14]:
subscript_dict = {'one_dimensional_subscript': (0, {'entry_1': 0, 'entry_2': 1, 'entry_3': 2}),
 'second_dimension_subscript': (1, {'column_1': 0, 'column_2': 1}),
 'third_dimension_subscript': (2, {'depth_1': 0, 'depth_2': 1})}

In [13]:
array = model.components.stock_a()
array

array([[[  1.99,   3.98],
        [  3.98,   8.95]],

       [[  5.97,   6.98],
        [  7.96,   6.96]],

       [[  9.95,   5.95],
        [ 11.94,   9.94]]])

In [15]:
for family, subscripts in subscript_dict.iteritems():
    

second_dimension_subscript
one_dimensional_subscript
third_dimension_subscript


In [12]:
array.flatten()

array([  1.99,   3.98,   3.98,   8.95,   5.97,   6.98,   7.96,   6.96,
         9.95,   5.95,  11.94,   9.94])

# prototype

In [ ]:
def get_array_info(subs, dictofsubs):
    """
    Returns information needed to create and access members of the numpy array
    based upon the string names given to them in the model file.

    Parameters
    ----------
    subs : Array of strings of subscripts

    returns
    -------
    A dictionary of the dimensions associating their names with their numpy indices
    directory = {'dimension name 1':0, 'dimension name 2':1}
    
    A list of the length of each dimension. Equivalently, the shape of the array:
    shape = [5,4]

    """
    
    for 
    
    # Todo: rework this to account for subarrays
    # todo: make this elementstr or something
    position=[]
    elements=element.replace('!','').replace('','').split(',')
    for element in elements:
        if element in dictofsubs.keys():
            position.append(len(dictofsubs[element]))
        else:
            for d in dictofsubs.itervalues():
                try:
                    (d[element])
                except: pass
                else:
                    position.append(len(d))

    return position



In [55]:
def hi:
    return 1

SyntaxError: invalid syntax (<ipython-input-55-ad431173dcc6>, line 1)

In [74]:
a = {'a':4, 'b':2, 'c':'r'}

a.keys()[a.values().index('r')]

'c'

In [75]:
a.keys()

['a', 'c', 'b']

In [76]:
a.values()

[4, 'r', 2]

In [ ]:
result = df['stock_a'].apply(lambda x: pd.Series(x.flatten()))

In [77]:
a = 'hi there'
[a]

['hi there']